In [1]:
from Crawling import Crawling

crawl = Crawling()
BGM_data = crawl.get_BGM_data()

START crawling
FIND 20 lists in 날림v CHANNEL
GET BGM Data from 날림v CHANNEL in 62.81s
END crawling


In [2]:
from Database import Database
DB = Database('alskfl')
DB.update_song(BGM_data)
DB.update_list(BGM_data)
DB.update_song_list(BGM_data)

DATABASE READY
USER : alskfl
Crawled BGM count: 1258
Before BGM count: 1230
New BGM count: 29
Deleted BGM count: 1
Table song updated
Crawled list count: 20
BGM 2022 02 (PLj63tGIo1sSpn_QiFEDqJHxxWskdXQI5j): inserted
Inserted list count: 1
Deleted list count: 0
Table list updated
BGM 2020 07 : 0 songs added
BGM 2020 08 : 0 songs added
BGM 2020 09 : 0 songs added
BGM 2020 10 : 0 songs added
BGM 2020 11 : 0 songs added
BGM 2020 12 : 0 songs added
BGM 2021 01 : 0 songs added
BGM 2021 02 : 0 songs added
BGM 2021 03 : 0 songs added
BGM 2021 04 : 0 songs added
BGM 2021 05 : 0 songs added
BGM 2021 06 : 0 songs added
BGM 2021 07 : 0 songs added
BGM 2021 08 : 0 songs added
BGM 2021 09 : 0 songs added
BGM 2021 10 : 0 songs added
BGM 2021 11 : 0 songs added
BGM 2021 12 : 0 songs added
BGM 2022 01 : 0 songs added
BGM 2022 02 : 72 songs added
927 20
172 20
903 20
944 20
243 20
18 20
1044 20
1077 20
1082 20
267 20
1090 20
552 20
1105 20
1109 20
1116 20
1123 20
862 20
731 20
1148 20
1155 20
450 20
398

![](youtube_playlist.png)

In [1]:
print('test')

test
